<a href="https://colab.research.google.com/github/hoky1227/Transformer_based-recommendation/blob/main/Instacart_Transformer_Pre_processing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import random
import time
import datetime
import csv
import os
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

In [ ]:
path = '/content/drive/MyDrive/Instacart/'
print(os.listdir(path))

def csvread(filename):
    return pd.read_csv(path + filename + '.csv')

# aisles = csvread('aisles')
# departments = csvread('departments')
# products = csvread('products')
# order_products_train = csvread('order_products__train')
# orders = csvread('orders')
# order_products_prior = csvread('order_products__prior')
order_products = pd.read_pickle(path + 'order_products.pkl')
ds = pd.read_pickle(path + 'InstacartCustomers.pkl')
# sample_submission = csvread('sample_submission')

['aisles.csv', 'departments.csv', 'products.csv', 'order_products__train.csv', 'orders.csv', 'order_products__prior.csv', 'sample_submission.csv', 'InstacartDetail.csv', '.ipynb_checkpoints', 'order_products.pkl', 'InstacartCustomers.pkl', 'InstacartOrders_sample.pkl']


In [ ]:
ds

,user,products
0,1,Organic Fuji Apples&&Bag of Organic Bananas&&H...
1,2,Organic Roasted Sliced Chicken Breast&&Articho...
2,3,"Crackers, Crispy, Cheddar&&Organic Baby Spinac..."
3,4,Original Orange Juice&&Extra-Dry Cava&&Organic...
4,5,"Semi-Soft Cheese, Ripened&&Red Raspberries&&Or..."
...,...,...
206204,206205,Hass Avocado&&Organic Whole String Cheese&&Ora...
206205,206206,Large Alfresco Eggs&&Gluten Free Apple Cinnamo...
206206,206207,Sparkling Water Grapefruit&&Organic Raspberrie...
206207,206208,Organic Whipping Cream&&Organic Lemon&&Peanut ...


In [ ]:
order_products

,order_id,product_id,user_id
0,431534,Organic Fuji Apples,1
1,431534,Bag of Organic Bananas,1
2,431534,Honeycrisp Apples,1
3,431534,Organic String Cheese,1
4,431534,Bartlett Pears,1
...,...,...,...
33819101,3154581,Diet Pepsi Pack,206209
33819102,3154581,Organic Diced Tomatoes,206209
33819103,3154581,Fat Free Cream Cheese,206209
33819104,3186442,Raisin Bran Cereal,206209


In [ ]:
def sortdf(data, column):
    data.sort_values(column, inplace=True)
    data.reset_index(inplace=True)
    data.pop('index')
    return data

In [ ]:
order_id = pd.DataFrame(columns=['order_id'])
order_id['order_id'] = pd.Series(order_products['order_id'].unique()).sample(frac=0.01).reset_index().pop('index')
print(len(order_id['order_id'].unique()))
order_id

33461


,order_id
0,95613
1,969347
2,4516
3,1930165
4,1527966
...,...
33456,2307371
33457,2458488
33458,2534301
33459,2900756


In [ ]:
order_sample = pd.merge(order_id, order_products, on='order_id', how='inner')
order_sample.pop('user_id')
order_sample

,order_id,product_id
0,95613,Sweet & Salty Nut Granola Bars Peanut
1,95613,Sea Salt Brown Rice Crackers
2,95613,Pub Mix
3,95613,Original Beef Jerky
4,95613,Salt & Pepper Pistachios
...,...,...
331819,2900756,Apple Honeycrisp Organic
331820,2900756,Unsalted Cultured Butter
331821,2900756,Organic Whole Milk
331822,2950922,Organic Low Fat Milk


In [ ]:
sortdf(order_sample, ['order_id'])

,order_id,product_id
0,289,Organic Sour Cream
1,289,Organic Gala Apples
2,289,Hass Avocados
3,289,Organic Raspberry Fruit Spread
4,289,Organic Red Cabbage
...,...,...
331819,3346047,Grape White/Green Seedless
331820,3346047,Plantain Chips
331821,3346047,Honey Oat Crunch Trail Mix Crunchy Granola Bars
331822,3346047,Firm Tofu


In [ ]:
df2 = pd.DataFrame(columns=['order', 'products'])
values = order_sample['order_id'].value_counts().index.sort_values()

for i, v in enumerate(tqdm(values)):
    df2.loc[i] = (v, '&&'.join(order_sample[order_sample['order_id'] == v]['product_id']))
df2

100%|██████████| 32752/32752 [05:17<00:00, 103.03it/s]


,order,products
0,289,All Natural Roasted Garlic Hummus&&Organic Cot...
1,378,Skim Milk&&Whole Vitamin D Milk
2,442,Total 2% Lowfat Plain Greek Yogurt&&Pitted Kal...
3,562,Crackers Cheddar Bunnies Snack Packs&&Organic ...
4,749,Organic Mushroom Base&&Chanterelle Mushrooms&&...
...,...,...
32747,3345838,Pomegranate on the Bottom Non-Fat Greek Yogurt...
32748,3345950,Veri Veri Teriyaki Marinade & Sauce&&Mild Ched...
32749,3345963,Unsweetened Organic Soy Milk&&Light Sour Cream...
32750,3346014,Half & Half&&Coke Zero&&Chicken Flavor Ramen N...


In [ ]:
df2.to_pickle(path + 'InstacartOrders_sample.pkl')

In [ ]:
df2 = pd.read_pickle(path + 'InstacartOrders_sample.pkl')
df2

,order,products
0,289,All Natural Roasted Garlic Hummus&&Organic Cot...
1,378,Skim Milk&&Whole Vitamin D Milk
2,442,Total 2% Lowfat Plain Greek Yogurt&&Pitted Kal...
3,562,Crackers Cheddar Bunnies Snack Packs&&Organic ...
4,749,Organic Mushroom Base&&Chanterelle Mushrooms&&...
...,...,...
32747,3345838,Pomegranate on the Bottom Non-Fat Greek Yogurt...
32748,3345950,Veri Veri Teriyaki Marinade & Sauce&&Mild Ched...
32749,3345963,Unsweetened Organic Soy Milk&&Light Sour Cream...
32750,3346014,Half & Half&&Coke Zero&&Chicken Flavor Ramen N...


In [ ]:
ds = df2.copy()
ds1 = ds['products'].str.split('&&')
# print(ds1[0][-1])
# print(ds1[0][-2])
ds1

0        [All Natural Roasted Garlic Hummus, Organic Co...
1                        [Skim Milk, Whole Vitamin D Milk]
2        [Total 2% Lowfat Plain Greek Yogurt, Pitted Ka...
3        [Crackers Cheddar Bunnies Snack Packs, Organic...
4        [Organic Mushroom Base, Chanterelle Mushrooms,...
                               ...                        
32747    [Pomegranate on the Bottom Non-Fat Greek Yogur...
32748    [Veri Veri Teriyaki Marinade & Sauce, Mild Che...
32749    [Unsweetened Organic Soy Milk, Light Sour Crea...
32750    [Half & Half, Coke Zero, Chicken Flavor Ramen ...
32751    [100% Pineapple Juice, Honey Oat Crunch Trail ...
Name: products, Length: 32752, dtype: object

In [ ]:
# # test = []
# for i in range(len(ds1)):
#     l = ds1[i][-1]
#     ds1[i].pop()
#     # test.append(l)

# display(ds1.head())
# # print(test[:5])
# ds1[0][-1]

In [ ]:
# ds1 = ds1.apply(lambda x: '&&'.join(map(str, x)))
# ds1.head()
# # ds1 = ds1.astype(str).str.replace('\[|\]|\'', '')
# # ds1 = pd.DataFrame(ds1)
# # ds1.head()

0    All Natural Roasted Garlic Hummus&&Organic Cot...
1                      Skim Milk&&Whole Vitamin D Milk
2    Total 2% Lowfat Plain Greek Yogurt&&Pitted Kal...
3    Crackers Cheddar Bunnies Snack Packs&&Organic ...
4    Organic Mushroom Base&&Chanterelle Mushrooms&&...
Name: products, dtype: object

In [ ]:
sequence_length = 5
step_size = 2

# [4, 4, 5, 5, 6, 6]이라면, 아래 함수를 통해
# [[4, 4, 5, 5], [5, 5, 6, 6]]으로 다시 만들어집니다.
def create_sequences(values, window_size, step_size):
    sequences = []
    start_index = 0
    while True:
        end_index = start_index + window_size
        seq = values[start_index:end_index]
        # 뒤에 남은 데이터가 window_size보다 작으면 중지합니다.
        if len(seq) < window_size:
            seq = values[-window_size:]
            # window_size와 같으면 list에 붙여줍니다.
            if len(seq) == window_size:
                sequences.append(seq)
            break
        sequences.append(seq)
        start_index += step_size
    return sequences


# ratings_data.movie_ids = ratings_data.movie_ids.apply(
#     lambda ids: create_sequences(ids, sequence_length, step_size)
# )

# ratings_data.ratings = ratings_data.ratings.apply(
#     lambda ids: create_sequences(ids, sequence_length, step_size)
# )

# del ratings_data["timestamps"]

In [ ]:
df_series1 = []

for i, v in enumerate(tqdm(ds1)):
    df_series1.append(create_sequences(v, sequence_length, step_size))
df_series1[:2]


100%|██████████| 32752/32752 [00:01<00:00, 23899.68it/s]


[[['All Natural Roasted Garlic Hummus',
   'Organic Cottage Cheese',
   'Probugs Strawnana Split Frozen Ypgurt',
   'Organic Red Bell Pepper',
   'Veggie Burger, Quarter Pound'],
  ['Probugs Strawnana Split Frozen Ypgurt',
   'Organic Red Bell Pepper',
   'Veggie Burger, Quarter Pound',
   'Spinach Peas & Pear Stage 2 Baby Food',
   'Country Dijon Mustard'],
  ['Veggie Burger, Quarter Pound',
   'Spinach Peas & Pear Stage 2 Baby Food',
   'Country Dijon Mustard',
   'Coconut Yogurt',
   'Wild Sweet Orange Herbal Tea'],
  ['Country Dijon Mustard',
   'Coconut Yogurt',
   'Wild Sweet Orange Herbal Tea',
   'Refresh Herbal Tea',
   'Organic Red Cabbage'],
  ['Wild Sweet Orange Herbal Tea',
   'Refresh Herbal Tea',
   'Organic Red Cabbage',
   'Organic Raspberry Fruit Spread',
   'Hass Avocados'],
  ['Organic Red Cabbage',
   'Organic Raspberry Fruit Spread',
   'Hass Avocados',
   'Organic Sour Cream',
   'Organic Gala Apples'],
  ['Hass Avocados',
   'Organic Sour Cream',
   'Organic Gal

In [ ]:
df_list = []
for v in tqdm(df_series1):
    for value in v:
        df_list.append(value)
df_list[:2]


100%|██████████| 32752/32752 [00:00<00:00, 731012.37it/s]


[['All Natural Roasted Garlic Hummus',
  'Organic Cottage Cheese',
  'Probugs Strawnana Split Frozen Ypgurt',
  'Organic Red Bell Pepper',
  'Veggie Burger, Quarter Pound'],
 ['Probugs Strawnana Split Frozen Ypgurt',
  'Organic Red Bell Pepper',
  'Veggie Burger, Quarter Pound',
  'Spinach Peas & Pear Stage 2 Baby Food',
  'Country Dijon Mustard']]

In [ ]:
label_list = []

for i in range(len(df_list)):
    l = df_list[i][-1]
    df_list[i].pop()
    label_list.append(l)

In [ ]:
df_list[:2]

[['All Natural Roasted Garlic Hummus',
  'Organic Cottage Cheese',
  'Probugs Strawnana Split Frozen Ypgurt',
  'Organic Red Bell Pepper'],
 ['Probugs Strawnana Split Frozen Ypgurt',
  'Organic Red Bell Pepper',
  'Veggie Burger, Quarter Pound',
  'Spinach Peas & Pear Stage 2 Baby Food']]

In [ ]:
label_list[:2]

['Veggie Burger, Quarter Pound', 'Country Dijon Mustard']

In [ ]:
df_new = pd.DataFrame(columns=['Detail', 'label'])
df_new['Detail'] = df_list
df_new['label'] = label_list
df_new

,Detail,label
0,"[All Natural Roasted Garlic Hummus, Organic Co...","Veggie Burger, Quarter Pound"
1,"[Probugs Strawnana Split Frozen Ypgurt, Organi...",Country Dijon Mustard
2,"[Veggie Burger, Quarter Pound, Spinach Peas & ...",Wild Sweet Orange Herbal Tea
3,"[Country Dijon Mustard, Coconut Yogurt, Wild S...",Organic Red Cabbage
4,"[Wild Sweet Orange Herbal Tea, Refresh Herbal ...",Hass Avocados
...,...,...
137172,"[Mixed Fruit Snacks, Tahitian Renewal Body Wa...",Blueberries
137173,"[100% Pineapple Juice, Honey Oat Crunch Trail ...",Golden Fishless Filet
137174,"[Plantain Chips, Grape White/Green Seedless, G...",Firm Tofu
137175,"[Golden Fishless Filet, Original No Pulp 100% ...",Organic Hass Avocado


In [ ]:
# m = 4
# for v in df_list:
#     a = len(v)
#     if a < m:
#         m = a
# m

In [ ]:
df_new['Detail'] = df_new['Detail'].apply(lambda x: '&&'.join(map(str, x)))
df_new

,Detail,label
0,All Natural Roasted Garlic Hummus&&Organic Cot...,"Veggie Burger, Quarter Pound"
1,Probugs Strawnana Split Frozen Ypgurt&&Organic...,Country Dijon Mustard
2,"Veggie Burger, Quarter Pound&&Spinach Peas & P...",Wild Sweet Orange Herbal Tea
3,Country Dijon Mustard&&Coconut Yogurt&&Wild Sw...,Organic Red Cabbage
4,Wild Sweet Orange Herbal Tea&&Refresh Herbal T...,Hass Avocados
...,...,...
137172,Mixed Fruit Snacks&&Tahitian Renewal Body Was...,Blueberries
137173,100% Pineapple Juice&&Honey Oat Crunch Trail M...,Golden Fishless Filet
137174,Plantain Chips&&Grape White/Green Seedless&&Go...,Firm Tofu
137175,Golden Fishless Filet&&Original No Pulp 100% F...,Organic Hass Avocado


In [ ]:
print(df_new.isnull().sum())
df_new.dropna(inplace=True)
df_new.isnull().sum()

Detail    0
label     0
dtype: int64


Detail    0
label     0
dtype: int64

In [ ]:
df_new.to_csv(path + 'Instacartdata_dup_sample.csv', mode='w')

In [ ]:
print(df_new.duplicated().sum())
df_new.drop_duplicates(inplace=True)
df_new.reset_index(inplace=True)
df_new.pop('index')
df_new

13093


,Detail,label
0,All Natural Roasted Garlic Hummus&&Organic Cot...,"Veggie Burger, Quarter Pound"
1,Probugs Strawnana Split Frozen Ypgurt&&Organic...,Country Dijon Mustard
2,"Veggie Burger, Quarter Pound&&Spinach Peas & P...",Wild Sweet Orange Herbal Tea
3,Country Dijon Mustard&&Coconut Yogurt&&Wild Sw...,Organic Red Cabbage
4,Wild Sweet Orange Herbal Tea&&Refresh Herbal T...,Hass Avocados
...,...,...
124079,Chicken Flavor Ramen Noodle Soup&&Ramen Noodle...,Presliced Everything Bagels
124080,Mixed Fruit Snacks&&Tahitian Renewal Body Was...,Blueberries
124081,100% Pineapple Juice&&Honey Oat Crunch Trail M...,Golden Fishless Filet
124082,Plantain Chips&&Grape White/Green Seedless&&Go...,Firm Tofu


In [ ]:
df_new.to_csv(path + 'Instacartdata_sample.csv', mode='w')